In [1]:
# Prueba del SODIOOOOO

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch
import pandas as pd


def map_category_to_number(categories):
    category_number_list = []
    for category in categories:
        if category == 'Apoyo Entusiasta':
            category_number_list.append(0)
        elif category == 'Críticas Severas':
            category_number_list.append(1)            
        elif category == 'Debate Económico':
            category_number_list.append(2)
        elif category == 'Preocupación por la Controversia':
            category_number_list.append(3)
        elif category == 'Comparaciones con Otros Políticos':
            category_number_list.append(4)
        elif category == 'Humor y Sarcasmo':
            category_number_list.append(5)
        
    return category_number_list

# Leer el archivo CSV
no_trained_comments = pd.read_csv("clean_comments_categorized.csv", names=['comment', 'category'])
trained_comments = pd.read_csv("train_comments.csv", names=['comment', 'category'])

no_trained_labels = map_category_to_number(no_trained_comments["category"].values[:10])
trained_labels = map_category_to_number(trained_comments["category"].values)

no_trained_comments = no_trained_comments['comment'].values[:10]
trained_comments = trained_comments['comment'].values



# Definir el conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, comments, labels_list):
        self.comments = comments
        self.labels_list = labels_list

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        return {'text': self.comments[idx], 'label': self.labels_list[idx]}

####Preba 1#####
# Cargar el tokenizer y el modelo preentrenado de BART
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
encoded_training_data = tokenizer.batch_encode_plus(trained_comments, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')
input_ids_train = encoded_training_data['input_ids']
attention_masks_train = encoded_training_data['attention_mask']
labels = torch.tensor(trained_labels)


encoded_eval_data = tokenizer.batch_encode_plus(no_trained_comments, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')
input_ids_eval = encoded_eval_data['input_ids']
attention_masks_eval = encoded_eval_data['attention_mask']




model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6, output_attentions=False, output_hidden_states=False)
# Define the training parameters
batch_size = 32
epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Train the model
for epoch in range(epochs):
    print(f"Epochs = {epoch}")
    model.train()
    for i in range(0, input_ids_train.size(0), batch_size):
        print(f"{i} - Dentro del segundo for")
        optimizer.zero_grad()
        print(f"{i} - Despues del zero_guard")
        outputs = model(input_ids_train[i:i+batch_size], attention_mask=attention_masks_train[i:i+batch_size], labels=labels[i:i+batch_size])
        print(f"{i} - Despues del outputs")
        loss = outputs.loss
        print(f"{i} - Despues del loss")
        loss.backward()
        print(f"{i} - Despues del backward")
        optimizer.steppredicted_categories = []
        

print(f"\n\n Entrenamiento terminado, comienza la evaluacion")

model.eval()
with torch.no_grad():
    outputs = model(input_ids_eval, attention_mask=attention_masks_eval)
    predicted_categories = torch.argmax(outputs[0], dim=1).flatten()

print(f"\n\nLas categorías predichas son: {predicted_categories}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/lucas/Documents/ITBA/Quinto_Ano_1-2_2024/NLP/TPE-NLP/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification 

Epochs = 0
Dentro del segundo for
Despues del zero_guard
Despues del outputs
Despues del loss
Despues del backward
Iteracion = 0
Dentro del segundo for
Despues del zero_guard
Despues del outputs
Despues del loss
Despues del backward
Iteracion = 32
Dentro del segundo for
Despues del zero_guard
Despues del outputs
Despues del loss
Despues del backward
Iteracion = 64
Dentro del segundo for
Despues del zero_guard
Despues del outputs
Despues del loss
Despues del backward
Iteracion = 96
Dentro del segundo for
Despues del zero_guard
Despues del outputs
Despues del loss
Despues del backward
Iteracion = 128


 Entrenamiento terminado, comienza la evaluacion


Las categorías predichas son: tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0])
